# 4. Ensembles

In [2]:
import numpy as np
import pandas as pd
import pymc3 as pm
import theano.tensor as tt
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, StackingRegressor

## Ensemble intro

### Wisdom of the crowds

- Wisdom of the crowds

### Ensemble methods

- What is ensemble learning?
- Voting classifiers
- Weak learners etc.

### Bagging

- What is bagging
- Bagging and pasting
- Out of bag evaluation

### Random bagging

- What is random bagging
- Random patches and subspaces

## 4.1 Random Forest

### Decisions trees

- Create a decision tree

### Random forest

- Bag a decision tree
- Random forrest regressor

### Extra trees

- Extremely randomised trees

### Random forrest optimisation

- Hyperparameter optimisation
- Grid search

### Tasks

Analyse the effect of the hyperparameters of the random forest, such as the number of estimators (or base models, i.e. the number of decision trees that are combined into the random forest). Look at the constructor of the RandomForestRegressor class to see what hyperparameters you can set. In your analysis, include the following plots and discussions but you may wish to add further analysis of your own:

1. Plot the relationship between a hyperparameter and the performance of the model.
2. Optimise the hyperparameter on a validation set.
3. Plot the trade-off between time taken for training and prediction performance.
4. What do you think is a good choice for the number of estimators on this dataset?
5. What is the effect of setting the maximum tree depth or maximum number of features?
6. Is the random forest interpretable? Are the decision trees that makeup the forest interpretable?

## 4.2 Stacking

### Stacking

- What is stacking

### Stacked regressors

- Stack Bayesian regression with decision trees

### Bayesian lienar regression recap

- Load in regressor from last task.

In [3]:
# load in bayesian regressor

### Stacking tasks

Bayesian linear regression and decision trees are two very different approaches to regression.  Ensemble methods can exploit such diversity between different methods to improve performance.  So now you will try combining the random forest  and  Bayesian  linear  regression  using stacking.

Scikit-learn  includes the  StackingRegressor  class  to  help  you  with  this.   In  the  report,  explain the stacking approach and describe your results,  making sure to cover the following points:

1. When does stacking improve performance over the individual models (e.g. try stacking with a random forest with 'maxdepth=10' and 'nestimators=10')?
2. What happens if we just take the mean prediction from our base models instead?
3. Use a DecisionTreeRegressor as the final estimator and visualise the tree to understand what stacking is doing.

### Boosting

- What is boosting

### Gradient boosting

- What is gradient boosting?

### XGBoost

- Implement XGBoost regression for housing data

### Stacking gradient

- Try stacking XGBoost regressors

## Ensemble Conclusion

### Comparison of random forest and stacking

- Comparison of random forest and stacking methods
  - Accuracy
  - Computational complexity
  - Data needed
  - Generalisation / ease of use